In [1]:
import numpy as np
import json
from datagenerator import min_len, max_len
num_of_diff_num = 10

#### Load the data

In [2]:
with open('./data.json', 'r') as f:
    data = json.load(f)

len(data)

150000

#### Preprocess the data

We will one hot encode the data. the input and output will have different length of one hot

In [3]:
x_train = np.zeros((len(data), max_len, num_of_diff_num)) # shape 20000x10. 10 is because the value in seq is 1 0 - 9
y_train = np.zeros((len(data), max_len)) # shape 20000x30. 30 is because max len of seq is 30
print(x_train.shape, y_train.shape)

(150000, 15, 10) (150000, 15)


In [4]:
seqs = []
page_faults = []
for i in range(len(data)):
    seqs.append(data[i]['seq'])
    page_faults.append(data[i]['page_faults'])

In [5]:
for i, seq in enumerate(seqs):
    for j, val in enumerate(seq):
        x_train[i, j, val] = 1

for i, pf in enumerate(page_faults):
    y_train[i, pf] = 1

#### Build the Model

In [12]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import RMSprop

model = Sequential()
model.add(LSTM(128, activation='relu', return_sequences=True, input_shape=(max_len, num_of_diff_num)))
model.add(Dropout(0.2))
model.add(LSTM(64, activation='relu', input_shape=(max_len,num_of_diff_num)))
model.add(Dropout(0.2))
model.add(Dense(max_len, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 15, 128)           71168     
_________________________________________________________________
dropout_5 (Dropout)          (None, 15, 128)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 15)                975       
Total params: 121,551
Trainable params: 121,551
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [8]:
history = model.fit(x_train, y_train, epochs=100, batch_size=256, validation_split=0.2)

Train on 120000 samples, validate on 30000 samples
Epoch 1/100
120000/120000 [==============================] - 82s - loss: 1.6742 - acc: 0.3110 - val_loss: 1.6042 - val_acc: 0.3301
Epoch 2/100
120000/120000 [==============================] - 81s - loss: 1.6048 - acc: 0.3297 - val_loss: 1.5703 - val_acc: 0.3613
Epoch 3/100
120000/120000 [==============================] - 83s - loss: 1.1756 - acc: 0.4784 - val_loss: 0.8802 - val_acc: 0.6046
Epoch 4/100
120000/120000 [==============================] - 85s - loss: 0.9111 - acc: 0.5948 - val_loss: 0.7959 - val_acc: 0.6494
Epoch 5/100
120000/120000 [==============================] - 81s - loss: 0.8188 - acc: 0.6353 - val_loss: 0.7247 - val_acc: 0.6776
Epoch 6/100
120000/120000 [==============================] - 85s - loss: 0.7604 - acc: 0.6641 - val_loss: 0.6738 - val_acc: 0.7048
Epoch 7/100
120000/120000 [==============================] - 85s - loss: 0.7220 - acc: 0.6835 - val_loss: 0.6727 - val_acc: 0.7110
Epoch 8/100
120000/120000 [=====

120000/120000 [==============================] - 81s - loss: 0.2447 - acc: 0.9059 - val_loss: 0.2565 - val_acc: 0.8987
Epoch 63/100
120000/120000 [==============================] - 81s - loss: 0.2407 - acc: 0.9080 - val_loss: 0.2425 - val_acc: 0.9056
Epoch 64/100
120000/120000 [==============================] - 81s - loss: 0.2355 - acc: 0.9119 - val_loss: 0.2292 - val_acc: 0.9110
Epoch 65/100
120000/120000 [==============================] - 80s - loss: 0.2318 - acc: 0.9116 - val_loss: 0.2914 - val_acc: 0.8826
Epoch 66/100
120000/120000 [==============================] - 80s - loss: 0.2281 - acc: 0.9137 - val_loss: 0.2488 - val_acc: 0.9036
Epoch 67/100
120000/120000 [==============================] - 81s - loss: 0.2224 - acc: 0.9162 - val_loss: 0.2425 - val_acc: 0.9035
Epoch 68/100
120000/120000 [==============================] - 80s - loss: 0.2200 - acc: 0.9173 - val_loss: 0.2303 - val_acc: 0.9108
Epoch 69/100
120000/120000 [==============================] - 80s - loss: 0.2186 - acc: 0

In [10]:
model.save('optNN_v1.h5')

ImportError: `save_model` requires h5py.